In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from dataclasses import dataclass
import json
import os
from pathlib import Path
from pprint import pprint
import sys

# seg_path = '../../lib/segmenter'
# if seg_path not in sys.path:
#     sys.path.append(seg_path)
# if '..' not in sys.path:
#     sys.path.append('..')
print(sys.path)

import shutil
import traceback
from typing import Union, Any, Optional
import yaml

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd

from sdp.ds.bop_data import read_models_info, read_scene_camera, read_scene_gt, \
    read_scene_gt_info
from sdp.ds.bop_dataset import BopDataset, AUGNAME_DEFAULT

from segm.model.factory import create_segmenter

['/Users/misha/prog/sdp/notebooks', '/Users/misha/prog/sdp', '/Users/misha/prog/lib/segmenter', '/Users/misha/miniconda3/envs/sdp_py310/lib/python310.zip', '/Users/misha/miniconda3/envs/sdp_py310/lib/python3.10', '/Users/misha/miniconda3/envs/sdp_py310/lib/python3.10/lib-dynload', '', '/Users/misha/miniconda3/envs/sdp_py310/lib/python3.10/site-packages']


In [4]:
DATA_PATH = Path(os.path.expandvars('$HOME/data'))
BOP_PATH = DATA_PATH / 'bop'
ITODD_SUBDIR = 'itodd'
TRAIN_SUBDIR = 'train_pbr'
ITODD_BOP_PATH = BOP_PATH / ITODD_SUBDIR
print(f'BOP path: {BOP_PATH}')

BOP path: /Users/misha/data/bop


In [10]:
ds = BopDataset.from_dir(BOP_PATH, ITODD_SUBDIR, shuffle=False)

Reading dataset from /Users/misha/data/bop/itodd/.sdp/v0.0.1


In [11]:
aug_name = None
img_size = 384
# aug_name = AUGNAME_DEFAULT

ov = ds.get_objs_view(1, out_size=img_size, aug_name=aug_name)

min_mask_ratio: 20671 --> 18066
min_bbox_dim_ratio: 18066 --> 16071


In [66]:
from dataclasses import asdict

import torch
from torch.hub import ENV_TORCH_HOME

from timm.models.helpers import load_pretrained, load_custom_pretrained
from timm.models.vision_transformer import default_cfgs
from timm.models.registry import register_model
from timm.models.vision_transformer import _create_vision_transformer

from segm.model.vit import VisionTransformer
from segm.model.utils import checkpoint_filter_fn
from segm.model.decoder import DecoderLinear
from segm.model.decoder import MaskTransformer
from segm.model.segmenter import Segmenter
import segm.utils.torch as ptu

if ENV_TORCH_HOME not in os.environ:
    os.environ[ENV_TORCH_HOME] = os.path.expandvars('$HOME/data/torch')
print(os.environ)

environ({'COMMAND_MODE': 'unix2003', 'CONDA_DEFAULT_ENV': 'sdp_py310', 'CONDA_EXE': '/Users/misha/miniconda3/bin/conda', 'CONDA_PREFIX': '/Users/misha/miniconda3/envs/sdp_py310', 'CONDA_PROMPT_MODIFIER': '(sdp_py310) ', 'CONDA_PYTHON_EXE': '/Users/misha/miniconda3/bin/python', 'CONDA_SHLVL': '2', 'HOME': '/Users/misha', 'LOGNAME': 'misha', 'MallocNanoZone': '0', 'OLDPWD': '/', 'ORIGINAL_XDG_CURRENT_DESKTOP': 'undefined', 'PATH': '/Users/misha/miniconda3/envs/sdp_py310/bin:/Users/misha/miniconda3/condabin:/Users/misha/apps/flutter/bin:/usr/local/smlnj/bin:/usr/local/bin:/System/Cryptexes/App/usr/bin:/usr/bin:/bin:/usr/sbin:/sbin:/Library/TeX/texbin:/Library/Apple/usr/bin:/var/run/com.apple.security.cryptexd/codex.system/bootstrap/usr/local/bin:/var/run/com.apple.security.cryptexd/codex.system/bootstrap/usr/bin:/var/run/com.apple.security.cryptexd/codex.system/bootstrap/usr/appleinternal/bin:/Users/misha/Library/Application Support/JetBrains/Toolbox/scripts', 'PWD': '/', 'SHELL': '/bin/z

In [64]:

def create_vit(model_cfg):
    model_cfg = model_cfg.copy()
    backbone = model_cfg.pop("backbone")

    normalization = model_cfg.pop("normalization")
    model_cfg["n_cls"] = 1000
    mlp_expansion_ratio = 4
    model_cfg["d_ff"] = mlp_expansion_ratio * model_cfg["d_model"]

    checkpoint_path = None
    if 'checkpoint_path' in model_cfg:
        checkpoint_path = model_cfg.pop('checkpoint_path')

    if backbone in default_cfgs:
        default_cfg = default_cfgs[backbone]
        default_cfg = asdict(default_cfg)
    else:
        default_cfg = dict(
            pretrained=False,
            num_classes=1000,
            drop_rate=0.0,
            drop_path_rate=0.0,
            drop_block_rate=None,
        )
    print('default_cfg:', default_cfg)
    default_cfg["input_size"] = (
        3,
        model_cfg["image_size"][0],
        model_cfg["image_size"][1],
    )
    model = VisionTransformer(**model_cfg)

    # if checkpoint_path:
    #     print(f'Reading encoder weights from {checkpoint_path}')
    #     state_dict = torch.load(checkpoint_path, map_location="cpu")
    #     filtered_dict = checkpoint_filter_fn(state_dict, model)
    #     model.load_state_dict(filtered_dict, strict=True)

    if backbone == "vit_base_patch8_384":
        path = os.path.expandvars("$TORCH_HOME/hub/checkpoints/vit_base_patch8_384.pth")
        state_dict = torch.load(path, map_location="cpu")
        filtered_dict = checkpoint_filter_fn(state_dict, model)
        model.load_state_dict(filtered_dict, strict=True)
    elif "deit" in backbone:
        load_pretrained(model, default_cfg, filter_fn=checkpoint_filter_fn)
    else:
        print('load_custom_pretrained')
        load_custom_pretrained(model, default_cfg)

    return model


def create_decoder(encoder, decoder_cfg):
    decoder_cfg = decoder_cfg.copy()
    name = decoder_cfg.pop("name")
    decoder_cfg["d_encoder"] = encoder.d_model
    decoder_cfg["patch_size"] = encoder.patch_size

    if "linear" in name:
        decoder = DecoderLinear(**decoder_cfg)
    elif name == "mask_transformer":
        dim = encoder.d_model
        n_heads = dim // 64
        decoder_cfg["n_heads"] = n_heads
        decoder_cfg["d_model"] = dim
        decoder_cfg["d_ff"] = 4 * dim
        decoder = MaskTransformer(**decoder_cfg)
    else:
        raise ValueError(f"Unknown decoder: {name}")
    return decoder


def create_segmenter(model_cfg):
    model_cfg = model_cfg.copy()
    decoder_cfg = model_cfg.pop("decoder")
    decoder_cfg["n_cls"] = model_cfg["n_cls"]

    encoder = create_vit(model_cfg)

    decoder = create_decoder(encoder, decoder_cfg)
    model = Segmenter(encoder, decoder, n_cls=model_cfg["n_cls"])

    return model


In [65]:
n_cls = 9
backbone = 'vit_tiny_patch16_384'
checkpoint_path = os.path.expandvars(f'$HOME/data/models/{backbone}.pth')

cfg = {
    'n_cls': n_cls,
    'backbone': backbone,
    'checkpoint_path': checkpoint_path,
    'image_size': (img_size, img_size),
    'channels': 9,
    'patch_size': 16,
    'd_model': 192,
    'n_heads': 3,
    'n_layers': 12,
    'normalization': 'vit',
    'distilled': False,
    'decoder': {
        'name': 'mask_transformer',
        'drop_path_rate': 0.1,
        'dropout': 0.1,
        'n_layers': 2
    }
}

model = create_segmenter(cfg)
model


default_cfg: {'tags': deque(['augreg_in21k_ft_in1k']), 'cfgs': {'augreg_in21k_ft_in1k': {'url': 'https://storage.googleapis.com/vit_models/augreg/Ti_16-i21k-300ep-lr_0.001-aug_none-wd_0.03-do_0.0-sd_0.0--imagenet2012-steps_20k-lr_0.03-res_384.npz', 'file': None, 'state_dict': None, 'hf_hub_id': 'timm/', 'hf_hub_filename': None, 'source': None, 'architecture': None, 'tag': None, 'custom_load': True, 'input_size': (3, 384, 384), 'test_input_size': None, 'min_input_size': None, 'fixed_input_size': True, 'interpolation': 'bicubic', 'crop_pct': 1.0, 'test_crop_pct': None, 'crop_mode': 'center', 'mean': (0.5, 0.5, 0.5), 'std': (0.5, 0.5, 0.5), 'num_classes': 1000, 'label_offset': None, 'label_names': None, 'label_descriptions': None, 'pool_size': None, 'test_pool_size': None, 'first_conv': 'patch_embed.proj', 'classifier': 'head', 'license': None, 'description': None, 'origin_url': None, 'paper_name': None, 'paper_ids': None, 'notes': None}}, 'is_pretrained': True}


No pretrained weights exist for this model. Using random initialization.


load_custom_pretrained
load_from = , pretrained_loc = 


Segmenter(
  (encoder): VisionTransformer(
    (patch_embed): PatchEmbedding(
      (proj): Conv2d(9, 192, kernel_size=(16, 16), stride=(16, 16))
    )
    (dropout): Dropout(p=0.1, inplace=False)
    (blocks): ModuleList(
      (0-11): 12 x Block(
        (norm1): LayerNorm((192,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((192,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (qkv): Linear(in_features=192, out_features=576, bias=True)
          (attn_drop): Dropout(p=0.1, inplace=False)
          (proj): Linear(in_features=192, out_features=192, bias=True)
          (proj_drop): Dropout(p=0.1, inplace=False)
        )
        (mlp): FeedForward(
          (fc1): Linear(in_features=192, out_features=768, bias=True)
          (act): GELU(approximate='none')
          (fc2): Linear(in_features=768, out_features=192, bias=True)
          (drop): Dropout(p=0.1, inplace=False)
        )
        (drop_path): Identity()
      )
    )
    (norm):

In [34]:
# print(list(default_cfgs.keys()))
dcfg = default_cfgs[backbone]
print(type(dcfg))

<class 'timm.models._pretrained.DefaultCfg'>


In [38]:
from dataclasses import asdict
asdict(dcfg)

{'tags': deque(['augreg_in21k_ft_in1k']),
 'cfgs': {'augreg_in21k_ft_in1k': {'url': 'https://storage.googleapis.com/vit_models/augreg/Ti_16-i21k-300ep-lr_0.001-aug_none-wd_0.03-do_0.0-sd_0.0--imagenet2012-steps_20k-lr_0.03-res_384.npz',
   'file': None,
   'state_dict': None,
   'hf_hub_id': 'timm/',
   'hf_hub_filename': None,
   'source': None,
   'architecture': None,
   'tag': None,
   'custom_load': True,
   'input_size': (3, 384, 384),
   'test_input_size': None,
   'min_input_size': None,
   'fixed_input_size': True,
   'interpolation': 'bicubic',
   'crop_pct': 1.0,
   'test_crop_pct': None,
   'crop_mode': 'center',
   'mean': (0.5, 0.5, 0.5),
   'std': (0.5, 0.5, 0.5),
   'num_classes': 1000,
   'label_offset': None,
   'label_names': None,
   'label_descriptions': None,
   'pool_size': None,
   'test_pool_size': None,
   'first_conv': 'patch_embed.proj',
   'classifier': 'head',
   'license': None,
   'description': None,
   'origin_url': None,
   'paper_name': None,
   'pap